In [1]:
import tdwf
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt 
import numpy as np
import scipy.optimize as so
import math
# === PARALLEL ===
import multiprocessing as mp


# === PARALLEL ===
# Worker di fit (solo CPU; nessun accesso all'hardware o a matplotlib)
def _fit_one(args):
    findex, ar, ff, tvals, ch1vals, ch2vals = args

    def fitfunc(x, A, phi, offset):
        return A * np.cos(2*np.pi*ff*x + phi) + offset

    pp1, _ = so.curve_fit(fitfunc, tvals, ch1vals, p0=[1, 0, 0])
    pp2, _ = so.curve_fit(fitfunc, tvals, ch2vals, p0=[1, 0, 0])

    # Aggiustamento fase se ampiezza negativa
    if pp1[0] < 0:
        pp1[0] *= -1
        pp1[1] += np.pi
    if pp2[0] < 0:
        pp2[0] *= -1
        pp2[1] += np.pi

    A_ratio = pp2[0] / pp1[0]
    phi = (pp2[1] - pp1[1] + np.pi) % (2*np.pi) - np.pi
    return (findex, ar, A_ratio, phi)


if __name__ == "__main__":
    # ==========================================================================
    #  Parametri dello script
    nper = 2            # numero periodi usati per la stima   
    npt = 8192          # numero MASSIMO di punti acquisiti
    nf = 200            # numero di frequenze nello sweep da f0 a f1   
    f0 = 50        
    f1 = 500e3      
    flag_return = False  # A/R? o solo andata?
    flag_show = True    # Visualizza i segnali nel tempo? (è uguale...)
    # vettore delle frequenze
    fv = np.logspace(np.log10(f0), np.log10(f1), nf) 

    # ==========================================================================
    #  Configurazione base AD2 (più parametri impostati dopo)
    ad2 = tdwf.AD2()
    ad2.vdd = 3
    ad2.vss = -3
    ad2.power(True)

    wavegen = tdwf.WaveGen(ad2.hdwf)
    wavegen.w1.ampl = 0.02
    wavegen.w1.func = tdwf.funcSine 
    wavegen.w1.start()
    scope = tdwf.Scope(ad2.hdwf)
    scope.ch1.rng = 5
    scope.ch2.rng = 50
    scope.ch1.avg = True
    scope.ch2.avg = True

    # ==========================================================================
    #   Esecuzione...

    # [1] Crea plot
    fig, [[ax1, ax3], [ax2, ax4]] = plt.subplots(2,2,figsize=(12,6),                                
        gridspec_kw={'width_ratios': [1, 2]})
    fig.canvas.manager.set_window_title('Spazzata frequenza')
    # [2] Preparazione dataset
    flag_first = True                # primo plot diverso dai seguenti (update)
    Am = np.full((nf, 2), np.nan)    # pre-allocazione dati (nan non sono plottati!) 
    phim = np.full((nf, 2), np.nan)  # pre-allocazione dati

    # === PARALLEL ===
    # Contenitore per i dati grezzi delle misure (da elaborare in parallelo)
    # Ogni elemento: (findex, ar, ff, tvals, ch1vals, ch2vals)
    tasks = []

    # [3] Ciclo misura: SOLO acquisizione (nessuna stima/fit qui)
    for ar in range(2 if flag_return else 1): # Ciclo A/R
        for ii in range(len(fv)):  # Ciclo frequenze
            # [3a] calcolo frequenza
            if ar==0: # caso A(ndata)
                findex = ii
            else:     # caso R(itorno)
                findex = len(fv)-ii-1
            # Frequenza attuale
            ff = fv[findex]
            # [3b] stima parametri di sampling
            df = math.ceil(100e6*nper/(npt*ff))
            scope.fs = 100e6/df
            scope.npt = int(scope.fs*nper/ff)
            #  Ribadiamo il trigger... 
            scope.trig(True, hist = 0.01)
            wavegen.w1.freq = ff 
            # [3c] Campionamento
            scope.sample()

            # === PARALLEL ===
            # Copia dei dati grezzi per il fit parallelo
            tvals  = scope.time.vals.copy()
            ch1val = scope.ch1.vals.copy()
            ch2val = scope.ch2.vals.copy()
            tasks.append((findex, ar, ff, tvals, ch1val, ch2val))

            # [3f] Aggiornamento plots (SOLO tempo, per non dipendere dai fit)
            if flag_first:
                flag_first = False
                if flag_show:
                    hp1, = ax1.plot(1000*scope.time.vals, scope.ch1.vals, "-", label="Ch1", color="tab:orange")
                    hp2, = ax2.plot(1000*scope.time.vals, scope.ch2.vals, "-", label="Ch2", color="tab:blue")
                    ax1.grid(True)
                    ax2.grid(True)
                    ax1.set_xticklabels([])
                    ax1.set_ylabel("Ch1 [V]", fontsize=15)
                    ax2.set_xlabel("Time [msec]", fontsize=15)
                    ax2.set_ylabel("Ch2 [V]", fontsize=15)
                    ax1.set_xlim([0, nper/ff])
                    ax2.set_xlim([0, nper/ff])
                    ax1.set_title(f"Starting")
                # Prepara i placeholder dei grafici Bode (si aggiorneranno DOPO i fit)
                hp3A, = ax3.loglog(fv, Am[:, 0], ".", markerfacecolor = "none", label="Amp go", color="tab:orange")
                hp4A, = ax4.semilogx(fv, phim[:, 0], ".",  markerfacecolor = "none", label="phi go", color="tab:orange")
                if flag_return:
                    hp3R, = ax3.loglog(fv, Am[:, 1], "v",  markerfacecolor = "none", label="Amp return", color="tab:blue")
                    hp4R, = ax4.semilogx(fv, phim[:, 1], "v",  markerfacecolor = "none", label="phi return", color="tab:blue")
                ax3.grid(True)
                ax4.grid(True)            
                ax3.set_xticklabels([])
                ax3.yaxis.tick_right()
                ax3.yaxis.set_label_position('right')
                ax3.set_ylabel("Gain [pure]", fontsize=15)
                ax4.set_xlabel("Freq [Hz]", fontsize=15)
                ax4.yaxis.tick_right()
                ax4.yaxis.set_label_position('right')
                ax4.set_yticks([-np.pi,-np.pi/2,0,np.pi/2,np.pi])
                ax4.set_yticklabels(["-180","-90","0","+90","+180"])
                ax4.set_ylabel("Phase [deg]", fontsize=15)
                ax3.legend()
                ax4.legend()
                plt.tight_layout()
                plt.show(block=False)    
            else:
                # Aggiorna titolo/limiti del tempo (facoltativo, non impatta la parallelizzazione)
                if ff > 1e3:
                    title = f"{ff/1e3:.1f}kHz"
                else:
                    title = f"{ff:.1f}Hz"
                if scope.fs > 1e6:
                    ax1.set_title(title+f" [{scope.npt:d}pts @ {scope.fs/1e6:.1f}MSa/s, ]")
                elif scope.fs > 1e3:
                    ax1.set_title(title+f" [{scope.npt:d}pts @ {scope.fs/1e3:.1f}kSa/s, ]")
                else:
                    ax1.set_title(title+f" [{scope.npt:d}pts @ {scope.fs:.1f}Sa/s, ]")
                if flag_show:
                    hp1.set_xdata(1000*scope.time.vals)
                    hp1.set_ydata(scope.ch1.vals)
                    hp2.set_xdata(1000*scope.time.vals)
                    hp2.set_ydata(scope.ch2.vals)
                    ax1.set_xlim([-500*nper/ff, +500*nper/ff])
                    ax2.set_xlim([-500*nper/ff, +500*nper/ff])
                    mi = scope.ch2.vals.min()
                    ma = scope.ch2.vals.max()
                    dm = ma-mi
                    m0 = (ma+mi)/2
                    ax2.set_ylim([m0-0.6*dm,m0+0.6*dm])
                fig.canvas.draw()
                fig.canvas.flush_events()

    # === PARALLEL ===
    # FASE 2: elaborazione in parallelo (fit)
    # Nota: nessun accesso all'hardware o a matplotlib nei worker
    with mp.Pool(processes=mp.cpu_count()) as pool:
        # imap_unordered per sfruttare meglio i core; ricomponiamo poi per indice
        for findex, ar, A_ratio, phi in pool.imap_unordered(_fit_one, tasks):
            Am[findex, ar] = A_ratio
            phim[findex, ar] = phi

    # === PARALLEL ===
    # Aggiorna i grafici Bode una volta conclusi i fit
    if 'hp3A' in locals():
        hp3A.set_ydata(Am[:, 0])
        hp4A.set_ydata(phim[:, 0])
        if 'hp3R' in locals() and flag_return:
            hp3R.set_ydata(Am[:, 1])
            hp4R.set_ydata(phim[:, 1])

        ax3.set_xlim([f0, f1])
        ax4.set_xlim([f0, f1])

        # Limiti Y robusti (ignorando NaN)
        if np.any(np.isfinite(Am[:, 0])):
            y_min = 0.5 * np.nanmin(Am[:, 0])
            y_max = 2.0 * np.nanmax(Am[:, 0])
            if np.isfinite(y_min) and np.isfinite(y_max) and (y_max > y_min) and (y_min > 0):
                ax3.set_ylim([y_min, y_max])

        fig.canvas.draw()
        fig.canvas.flush_events()

    # Salvataggio risultati come prima
    data = np.column_stack((fv, Am, phim))
    np.savetxt("AD8031_Gain1e3.txt", data)

    # ---------------------------------------
    ad2.close()


Digilent WaveForms SDK versione 3.22.2
Dispositivo #1 [SN:210321B5D136, hdwf=1] connesso!
Configurazione #1


: 